# Profiling LLaMA 4 Inference with vLLM


## Introduction

Profiling is essential for understanding the performance bottlenecks in large language model inference pipelines. In this tutorial, we walk through the process of profiling the **LLaMA 4 Scout 17B-16E-Instruct** model using the `vLLM` framework on AMD GPUs with ROCm. We'll capture detailed kernel traces and later demonstrate how to visualize them using perfetto.

## Prerequisites
Before starting this tutorial, ensure you have the following:

- Access to the gated model **LLaMA 4 Scout 17B-16E-Instruct**
- Access to **Perfetto UI** 

### Hardware

* **AMD GPUs**: Ensure you are using an AMD GPU, such as the Instinct™ MI300X or Radeon Pro W7900, with ROCm support and that your system meets the [official requirements](https://rocm.docs.amd.com/projects/install-on-linux/en/latest/reference/system-requirements.html).

### Software

* **ROCm 6.3, 6.4**: Install and verify ROCm by following the [ROCm install guide](https://rocm.docs.amd.com/projects/install-on-linux/en/latest/install/quick-start.html). Verify your ROCm installation by running this command:

   ``` bash
   rocm-smi
   ```
   This command produces a table similar to the one below:

   ![rocm-smi](../assets/rocm-smi.png)

* **Docker**: Ensure Docker is installed and configured correctly. See the [Docker installation guide](https://docs.docker.com/get-docker/) for more information.

### 1. Pull the Docker image

Ensure your system meets the [system requirements](https://rocm.docs.amd.com/projects/install-on-linux/en/latest/reference/system-requirements.html).

Pull the Docker image required for this tutorial:

``` bash
docker pull rocm/vllm-dev:main
```

### 2. Launch the Docker container

Launch the Docker container in a terminal on your server and map the necessary directories.

``` bash
docker run -it --rm \
  --network=host \
  --device=/dev/kfd \
  --device=/dev/dri \
  --group-add=video \
  --ipc=host \
  --cap-add=SYS_PTRACE \
  --security-opt seccomp=unconfined \
  --shm-size 8G \
  -v $(pwd):/workspace \
  -w /workspace/notebooks \
  rocm/vllm:latest
```

**Note**: This command mounts the current directory to the `/workspace` directory in the container. Ensure the notebook file is either copied to this directory before running the Docker command or uploaded into the Jupyter Notebook environment after it starts. Save the token or URL provided in the terminal output to access the notebook from your web browser. You can download this notebook from the [AI Developer Hub GitHub repository](https://github.com/ROCm/gpuaidev).

### Install and launch Jupyter

Inside the Docker container, install Jupyter using the following command:

``` bash
pip install jupyter
```

Start the Jupyter server:

``` bash
jupyter-lab --ip=0.0.0.0 --port=8888 --no-browser --allow-root
```

## Step by step process

### Step 1: Hugging Face Login
Provide your Hugging Face token

You'll require a Hugging Face API token to access Llama-3.1. Generate your token at [Hugging Face Tokens](https://huggingface.co/settings/tokens) and request access for [LLaMA 4 Scout 17B-16E-Instruct](https://huggingface.co/meta-llama/Llama-4-Scout-17B-16E-Instruct). Tokens typically start with "hf_". 

Run the following interactive block in your Jupyter notebook to set up the token:

**Note**: Uncheck the "Add token as Git credential" option.

In [ ]:
from huggingface_hub import notebook_login, HfApi

# Prompt the user to log in
notebook_login()

Verify that your token was accepted correctly:

In [ ]:
from huggingface_hub import HfApi

try:
    api = HfApi()
    user_info = api.whoami()
    print(f"Token validated successfully! Logged in as: {user_info['name']}")
except Exception as e:
    print(f"Token validation failed. Error: {e}")

### Step 2: Start the vLLM Server with profiler configs
Open a new terminal tab inside your JupyterLab session.
In this new terminal, run the following commands and keep the terminal open.
```shell
mkdir -p /profile
export VLLM_TORCH_PROFILER_DIR=/profile

# Start the vLLM server with standard configs
RCCL_MSCCL_ENABLE=0 \
VLLM_USE_V1=1 \
VLLM_WORKER_MULTIPROC_METHOD=spawn \
VLLM_USE_MODELSCOPE=False \
VLLM_USE_TRITON_FLASH_ATTN=0 \
vllm serve meta-llama/Llama-4-Scout-17B-16E-Instruct \
    --disable-log-requests \
    -tp 8 \
    --max-num-seqs 64 \
    --no-enable-prefix-caching \
    --max_num_batched_tokens=320000 \
    --max-model-len 32000

```

### Step 3: Run the Benchmark and Capture the Trace

With the server running, trigger a synthetic benchmark request to generate traffic and collect profiling data:

In [ ]:
!vllm bench serve \
    --model meta-llama/Llama-4-Scout-17B-16E-Instruct \
    --dataset-name random \
    --random-input-len 2000 \
    --random-output-len 10 \
    --max-concurrency 64 \
    --num-prompts 64 \
    --ignore-eos \
    --percentile_metrics ttft,tpot,itl,e2el \
    --profile

Make sure to add the —profile flag, which basically starts a profiler before the benchmark starts and then end and write the trace once the profiler ends.

***NOTE*** Once the above cell execution is completed, stop the vLLM server you started in the other terminal by pressing `ctrl` + `c` at this point.



### Step 4 : Visualize the Trace Using [ui.perfetto.dev](https://ui.perfetto.dev/)

Once the trace is generated unzip it and it will be of the json form, it's time to visualize it using Perfetto, a powerful trace viewer built for large-scale profiling data. This helps uncover latency bottlenecks, CPU–GPU overlap, and kernel-level inefficiencies in your inference pipeline.

- Go to: [https://ui.perfetto.dev](https://ui.perfetto.dev/)
- Click **"Open trace file"**
- Upload the `.json` file

Once you open the trace it will look something like this:

![image_0](../assets/image_0.png)


#### Understanding Prefill and Decode Timelines

Let’s now zoom into the trace and interpret what each slice reveals about LLaMA 4’s execution stages. Here's a close-up of two important process timelines captured in Perfetto:

> Our focus here is on the first two tracks: python3 3906 and python3 2.
> 

---

#### `python3 3906` – Asynchronous CPU Calls

The **orange slice** under `python3 3906` shows the execution of high-level model code on the CPU. It contains:

- Calls like `execute_model`, `forward`, and `hipMemcpyWithStream`
- PyTorch internals like `aten::to`, `aten::copy_`
- The actual forward pass for `Llama4ForCausalLM` and memory transfers

This slice reflects the CPU-side orchestration of inference — from input preparation to dispatching kernels to the GPU.

---

#### `python3 2` – GPU Kernel Timeline

The **pink slice** under `python3 2` is where GPU kernel execution is visualized. This slice represents actual compute work being done by the GPU after CPU enqueues the tasks.

Here’s the key insight:

- There is a **clear gap** between two bursts of kernel execution.
- This gap separates two distinct phases:
    - **Before the gap**: **Prefill** — where the initial prompt is encoded and attention/cache states are populated.
    - **After the gap**: **Decode** — where the model generates tokens, typically using cached key/value tensors.

#### Understanding Kernel Timelines:

![image](../assets/image_1.png)


When you expand the `python3 2` timeline in Perfetto, you’ll see two distinct GPU streams representing different types of operations executed on the device:

> This zoomed-in view helps us distinguish between computation kernels and communication operations.
> 

---

#### Stream `3 3`: All GPU Kernels

Stream `3 3` is the **primary compute stream**, where **all GPU kernel executions** are scheduled. This includes:

- `MatMul` / GEMM operations
- Fused MLP and attention blocks
- Positional encodings, LayerNorms
- Any fused or element-wise kernels

This stream is densely packed, showing the **bulk of model inference** in action. The rhythm and spacing of these kernels help diagnose things like:

- Load balancing across tensor parallel ranks
- Gaps between kernel launches
- Prefill vs Decode phases (based on density before/after gaps)

---

#### Stream `3 8`: AllGather Kernels

Stream `3 8` is used specifically for **AllGather operations**, which are part of the tensor parallel communication process. These kernels:

- Synchronize activations across devices in multi-GPU setups
- Typically occur **between layer boundaries**
- Are crucial in `tp=8` setups for syncing partial outputs across 8 shards

### Zooming In: Analyzing the Attention Forward Kernel

Attention is at the heart of all Transformer-based language models — and LLaMA 4 is no exception. Profiling its **attention forward kernel** gives us critical insights into computational efficiency.

Let’s dive into the trace and inspect it at the kernel level.

---

#### Zoom Into the Kernel Timeline

- Navigate to `python3 2` → `stream 3 3` in the Perfetto UI.
- **Use your mouse scroll** or drag to zoom into a cluster of dense kernels.
- Hover over one of the prominent kernels — you should see a label like `_fwd_kernel`.

Here’s an example image from our trace:

![image](../assets/image_2.png)


---

#### Understanding the Kernel Slice

From the **details panel** below the trace view, we can extract:

| Field | Value |
| --- | --- |
| **Name** | `_fwd_kernel` |
| **Category** | `kernel` |
| **Stream** | `stream 3 (3)` |
| **Process** | `python3 (2)` |
| **Duration** | `788µs 256ns` |
| **Launch Delay** | `~819µs` from CPU call |

This kernel is part of the **multi-head attention forward pass**, one of the most compute-heavy operations in inference.

---

#### Trace It Back to the CPU

Inside the kernel details near proceeding flows, you’ll find `hipModuleLaunchKernel` 

Clicking this will **jump back to the CPU thread** (`python3 3906`) in blue in the below picture that issued this kernel launch. This feature is incredibly useful for:

- Mapping GPU ops to their Python or C++ call stack
- Identifying bottlenecks in dispatch or synchronization
- Understanding how long the CPU takes to enqueue work on the GPU

![image](../assets/image_3.png)


Following the above process you can track which kernel takes how much time in both prefill and decode, one of our short summary are mentioned in the below chart.

![image](../assets/image_4.png)

### Step 7 Programmatic Analysis:  Extracting GPU Kernel Timeline with Python

To go beyond visual inspection, you can also parse the trace programmatically to list all GPU kernels and their durations. This is helpful when you're tracking:

- Kernel launch patterns
- Duration spikes
- Gaps or anomalies in execution

Here’s a minimal Python script that reads the `trace.json` file and lists all GPU kernels sorted by start time:


Decompress the .gz trace file into trace.json.

In [ ]:
!gunzip -c /profile/$(ls /profile | head -n 1) > trace.json

Run the below cell which will lists all GPU kernels sorted by start time

In [ ]:

import json

# Load the trace
with open("trace.json", "r") as f:
    trace = json.load(f)

gpu_kernels = []

# Extract GPU kernel events
for event in trace["traceEvents"]:
    if event.get("ph") != "X":
        continue

    cat = event.get("cat", "").lower()
    name = event.get("name", "")
    start_time = event["ts"]
    duration = event.get("dur", 0)

    if "cuda" in cat or "kernel" in cat:
        gpu_kernels.append({
            "name": name,
            "start": start_time,
            "duration": duration
        })

# Print all GPU kernels with their durations
print(f"{'GPU Kernel':<60} {'Start (us)':<15} {'Duration (us)':<15}")
print("-" * 90)
for k in sorted(gpu_kernels, key=lambda x: x["start"]):
    print(f"{k['name']:<60} {k['start']:<15} {k['duration']:<15}")


## Conclusion:

In this tutorial, we walked through the **end-to-end process of profiling LLaMA 4 inference on AMD GPUs using vLLM**, starting from:

- Container setup with ROCm and vLLM
- Enabling and capturing detailed performance traces
- Visualizing CPU-GPU interactions with [Perfetto](https://ui.perfetto.dev/)
- Zooming into kernel-level activity for attention blocks
- Programmatically analyzing trace logs using Python